# Grafici Trasmittanza Spettrofotometro


In [6]:
import Functions_Tries.plotter as plotter
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.optimize import curve_fit
import pandas as pd
import numpy as np
from Functions_Tries.transmittance import Transmittance
from pathlib import Path


plt.style.use("seaborn")

/tmp/ipykernel_2331/684767676.py:12: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn")


# Il codice seguente serve al fit dello spessore dagli spettri ottenuti dallo spettrofotometro

- Usiamo la libreria ```numpy``` per avere la matematica corretta e dei contenitori efficenti.
- Usiamo la libreria ```scipy.interpolate``` per avere le curve di Bezier per interpolare i dati ottenuti dal Johnson
- Usiamo la libreria ```scipy.optimize``` per fittare gli spettri e ottenere il valore di spessore e l'errore
- Usiamo la libreria ```pandas``` per leggere i file che contengono gli spettri e per l'output
- Stefano ha scritto il modulo ```transmittance``` per avere le funzioni su cui fare il fit e per fare il primo test di fattibilità sui fit
- Usiamo il modulo ```pathlib``` per gestire i Path dei vari file in maniera automatizzata
- Usiamo la libreria ```matplotlib``` per graficare gli spettri

## TODO

- [ ] Aggiungere il salvataggio di tutti i risultati su un file
- [ ] Fare i grafici automatici, salvandoli in una cartella separata Graph

In [7]:
np.set_printoptions(precision=2)

dir = Path("./data/")

# leggo dati del johnny
john = pd.read_csv("./data/book_data/Johnson.csv")
k_spl_john = CubicSpline(john["wl"], john["k"])
n_spl_john = CubicSpline(john["wl"], john["n"])

# Inizializziamo la classe che contiene le transmittance per i successivi fit
Trans = Transmittance(n = n_spl_john, k = k_spl_john)

# Funzione che va a fare i fit in maniera automatica a partire dai dati ottenuti
def optimizer(path):
    df=pd.read_csv(path)
    popt, pcov = curve_fit(
        Trans.beer_lambert,
        df[(df["lambda"] < 800e-9) & (df["lambda"] > 300e-9)]["lambda"],
        df[(df["lambda"] < 800e-9) & (df["lambda"] > 300e-9)]["polished"],
        p0=60e-9,
    )
    return popt, pcov


# Funzione che itera su tutti i file contenuti nella cartella "./data"
def iterazione(path):
    data =[i for i in path.iterdir() if i.match("*spettrofotometro")]
    # print(data)
    for i in data:
        subfolder = [j for j in i.iterdir() if j.is_dir()]
        for folder in subfolder:
            if folder.match("ELAB"):
                for file in folder.iterdir():
                    print(file)
                    print(optimizer(file))    
            else:
                print("La cartella non è Elab... gestiamo logica dopo")
                print(folder)
                iterazione_spettro(folder)

# In alcuni casi non si trova subito la cartella "ELAB", in quei casi bisogna scendere ancora di un livello
def iterazione_spettro(path):
    print("launching iterazione_spettro")
    print(path)
    elab = [i for i in path.iterdir() if i.match("ELAB")]
    print(elab)
    for i in elab:
        for file in i.iterdir():
            print(file)
            print(optimizer(file))

# Finalmente lanciamo la cazzo di funzione
iterazione(dir)
   

/home/pilo/miniconda3/envs/codeLab310/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


data/21-04_spettrofotometro/ELAB/AU-Vetro-Vetro-900-350-5nm_1.csv
(array([1.3e-08]), array([[8.92e-21]]))
data/21-04_spettrofotometro/ELAB/AU-PDMS-Vetro-PDMS-Vetro-900-350_1.csv
(array([1.67e-08]), array([[1.61e-20]]))
data/21-04_spettrofotometro/ELAB/Aria-Aria-900-350.csv
(array([-9.55e-19]), array([[inf]]))
data/21-04_spettrofotometro/ELAB/AU_vetro-Vetro-900-350.csv
(array([7.91e-08]), array([[1.06e-20]]))
data/21-04_spettrofotometro/ELAB/AU-Vetro-Vetro-900-350-5nm_2.csv
(array([1.42e-09]), array([[2.13e-21]]))
data/2-05_spettrofotometro/ELAB/Oro-Oro 1.csv
(array([3.84e-08]), array([[1.23e-20]]))
data/2-05_spettrofotometro/ELAB/Oror-oro 4_1.csv
(array([5.84e-08]), array([[1.44e-20]]))
data/2-05_spettrofotometro/ELAB/Oror-oro 3_1.csv
(array([5.14e-08]), array([[1.33e-20]]))
data/2-05_spettrofotometro/ELAB/Oror-oro 2_1.csv
(array([4.29e-08]), array([[1.33e-20]]))
data/2-05_spettrofotometro/ELAB/Aria-Aria_1- Assorb.csv
(array([4.89e-08]), array([[3.13e-19]]))
data/2-05_spettrofotometro/

/home/pilo/miniconda3/envs/codeLab310/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/home/pilo/miniconda3/envs/codeLab310/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/home/pilo/miniconda3/envs/codeLab310/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


(array([6.65e-08]), array([[1.06e-20]]))
data/5-05_spettrofotometro/vetrino2_spettrofotometro/ELAB/vetroAU-vetro_1_2.csv
(array([1.64e-08]), array([[3.11e-17]]))
data/5-05_spettrofotometro/vetrino2_spettrofotometro/ELAB/air_air_trasm_2.csv
(array([-9.55e-19]), array([[inf]]))
data/5-05_spettrofotometro/vetrino2_spettrofotometro/ELAB/vetroAU-vetro_4_2.csv
(array([4.47e-08]), array([[1.38e-20]]))
data/5-05_spettrofotometro/vetrino2_spettrofotometro/ELAB/vetroAU-vetro_5_2.csv
(array([3.34e-08]), array([[1.34e-20]]))
La cartella non è Elab... gestiamo logica dopo
data/5-05_spettrofotometro/vetrino1_spettrofotometro
launching iterazione_spettro
data/5-05_spettrofotometro/vetrino1_spettrofotometro
[PosixPath('data/5-05_spettrofotometro/vetrino1_spettrofotometro/ELAB')]
data/5-05_spettrofotometro/vetrino1_spettrofotometro/ELAB/vetroAU-vetro_5_1.csv
(array([3.34e-08]), array([[1.34e-20]]))
data/5-05_spettrofotometro/vetrino1_spettrofotometro/ELAB/vetroAU-vetro_2_1.csv
(array([6.65e-08]), array

/home/pilo/miniconda3/envs/codeLab310/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/home/pilo/miniconda3/envs/codeLab310/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
